In [ ]:
from mads_datasets import DatasetFactoryProvider, DatasetType
from mltrainer.preprocessors import PaddedPreprocessor

from mltrainer import Trainer
from torch import optim

from mltrainer import TrainerSettings, ReportTypes
from mltrainer.metrics import Accuracy
import torch

import sys 
import os
sys.path.append(os.path.abspath('../networks'))
sys.path.append(os.path.abspath('../dev'))

from RNN import ModelConfig

preprocessor = PaddedPreprocessor()

gesturesdatasetfactory = DatasetFactoryProvider.create_factory(DatasetType.GESTURES)
streamers = gesturesdatasetfactory.create_datastreamer(batchsize=32, preprocessor=preprocessor)
train = streamers["train"]
valid = streamers["valid"]

trainstreamer = train.stream()
validstreamer = valid.stream()

In [ ]:
accuracy = Accuracy()
loss_fn = torch.nn.CrossEntropyLoss()

config = ModelConfig(
    input_size=3, # vast
    hidden_size=128,
    num_layers=2,
    output_size=20, # vast
    dropout=0.2,
)

In [ ]:
from pathlib import Path

settings = TrainerSettings(
    epochs=10, 
    metrics=[accuracy],
    logdir=Path("gestures"),
    train_steps=len(train),
    valid_steps=len(valid),
    reporttypes=[ReportTypes.TOML, ReportTypes.TENSORBOARD],
    scheduler_kwargs={"factor": 0.5, "patience": 5},
    earlystop_kwargs = {
        "save": False, # save every best model, and restore the best one
        "verbose": True,
        "patience": 5, # number of epochs with no improvement after which training will be stopped
    }
)
settings

In [ ]:
from RNN import RecurrentNeuralNetwork

model = RecurrentNeuralNetwork(
    config=config,
)

trainer = Trainer(
    model=model,
    settings=settings,
    loss_fn=loss_fn,
    optimizer=optim.Adam,
    traindataloader=trainstreamer,
    validdataloader=validstreamer,
    scheduler=optim.lr_scheduler.ReduceLROnPlateau,
)

trainer.loop()

In [ ]:
from RNN import GRUWithAttention

model = GRUWithAttention(
    config=config,
)

trainer = Trainer(
    model=model,
    settings=settings,
    loss_fn=loss_fn,
    optimizer=optim.Adam,
    traindataloader=trainstreamer,
    validdataloader=validstreamer,
    scheduler=optim.lr_scheduler.ReduceLROnPlateau,
)

trainer.loop()

In [ ]:
from RNN import RecurrentNeuralNetworkWithAttention

model = RecurrentNeuralNetworkWithAttention(
    config=config,
)

trainer = Trainer(
    model=model,
    settings=settings,
    loss_fn=loss_fn,
    optimizer=optim.Adam,
    traindataloader=trainstreamer,
    validdataloader=validstreamer,
    scheduler=optim.lr_scheduler.ReduceLROnPlateau,
)

trainer.loop()